# Demo 5 — After-Action Replay & Doctrinal Clinic
## Learning from the Game — Closing the OODA Loop

**OODA Phase: Closing the loop — feeding lessons back into future Observe / Orient cycles**

**Purpose:** Given a completed game log (from Demo 2 or a hand-crafted transcript), a team of agents automatically reconstructs the decision timeline, identifies pivotal turning points, evaluates each key decision against doctrinal references, and produces a structured AAR report with "lessons observed." The user then drives an interactive interrogation — selecting decision points, asking "what if?", and watching agents generate counterfactual branches on the spot.

**Audience:** Researchers, exercise designers, training professionals, and those exploring AI-assisted analysis in synthetic wargaming environments.
**Primary outcome:** A structured AAR report (Observation → Discussion → Recommendation → Training Implication) produced by cooperating agents, with interactive counterfactual exploration driven by a human-in-the-loop facilitator.


> **Responsible AI & Scope Statement:** This research explores human-AI collaboration and explainable decision-support in fully synthetic, abstract environments. All scenarios, agents, and data are artificially generated. No real-world operational data, contingency plans, systems, or intelligence are used or represented. The prototype is intended exclusively for research, educational, and experimental purposes and does not constitute an operational model, validated planning tool, or source of decision authority. Doctrinal references are inspired by publicly available concepts and do not represent authoritative U.S. Navy doctrine.


## Azure Technologies Used in This Demo

This demo relies on several Azure services working together. If you are new to Azure, here is a brief overview of each technology and how it fits into the pipeline.

| Technology | Role in This Demo | Learn More |
|---|---|---|
| **Azure AI Foundry** | Hosts and serves the large language model (GPT-4o) that powers every agent in the AAR pipeline. Provides a unified inference endpoint so you can call models without managing GPU infrastructure. | [Azure AI Foundry documentation](https://learn.microsoft.com/azure/ai-foundry/) |
| **Azure AI Model Inference API** | The REST / SDK interface used by `AzureAIChatCompletionClient` to send prompts and receive completions from models deployed in Azure AI Foundry. | [Azure AI Model Inference API](https://learn.microsoft.com/azure/ai-foundry/model-inference/overview) |
| **Azure Key Vault** | Securely stores the API keys and endpoint URLs needed to call the model. Secrets are never hard-coded; instead they are fetched at runtime using `DefaultAzureCredential`. | [Azure Key Vault overview](https://learn.microsoft.com/azure/key-vault/general/overview) |
| **Azure Identity (`DefaultAzureCredential`)** | Provides a credential chain that works transparently in multiple environments — Managed Identity in Azure, `az login` locally, or environment variables in CI/CD — so no secret material ever appears in code. | [Azure Identity client library](https://learn.microsoft.com/python/api/overview/azure/identity-readme) |
| **Microsoft AutoGen** | An open-source multi-agent orchestration framework from Microsoft Research. This demo uses `autogen-agentchat` to create specialized agents (Narrator, Critic, etc.) and `autogen-ext[azure]` to connect them to Azure-hosted models. | [AutoGen documentation](https://microsoft.github.io/autogen/) |

> **Tip:** If you do not yet have an Azure subscription, you can create a [free Azure account](https://azure.microsoft.com/free/) to explore these services.


## What It Illustrates (Multi-Agent)

| Agent | Role | Unique Contribution |
|-------|------|---------------------|
| **Replay Narrator** | Ingests game log; reconstructs chronological decision narrative; identifies 3–5 most consequential decision points | Pivot detection — which decisions most changed the distribution of possible outcomes |
| **Doctrinal Critic** | Evaluates each decision against abstracted doctrinal knowledge base (principles inspired by publicly available naval warfare concepts, ROE frameworks) | Structured assessment: doctrine applied, consistency, justified deviation, doctrinal gaps exposed |
| **Counterfactual Branch** | Perturbs selected decisions and propagates changes forward through remaining turns | Interactive "what if?" exploration — reuses Demo 4 engine concept |
| **Lessons Learned Compiler** | Synthesizes all analysis into ODRI format (Observation, Discussion, Recommendation, Implication for Training) | Military-professional structured output familiar to AAR audiences |
| **Human Facilitator (HITL)** | Selects decision points, asks "what if?", challenges assessments, asks open-ended questions | Interactive facilitation — system adapts to user focus |
| **Explainability** | Produces transparent reasoning traces; identifies biases, confidence levels, and areas for human review | Ensures the entire analysis pipeline is auditable and inspectable |

**Success criteria:** Agents produce a timeline of pivotal decisions, evaluate each against abstracted doctrinal concepts, support interactive counterfactual exploration, produce transparent reasoning traces, and compile a structured lessons-learned document — all from a game log produced by Demo 2. All outputs are recommendations and analysis for human review, not directives.


## Demo Script (Presenter Guide)

1. **Intro (1 min):** "Demos 1–3 *played* the game. Demo 4 *challenged reasoning*. Now Demo 5 *extracts durable institutional knowledge*. This is the AAR that writes itself."
2. **Load game log (30 sec):** Show the JSON from Demo 2. "This is the raw encounter data — five turns of gray-zone escalation with scored actions, escalation tracking, and agent reasoning."
3. **Phase 1 — Replay (2 min):** Run the Narrator. "Watch it reconstruct the decision timeline and identify the 3–5 moments where the outcome pivoted."
4. **Phase 2 — Doctrinal Critique (2 min):** "Now the Doctrinal Critic evaluates each decision against abstracted naval warfare principles and ROE concepts inspired by publicly available frameworks. This is what makes the AAR *structured and traceable* rather than anecdotal."
5. **Phase 3 — Counterfactual (2 min):** Select a decision point and ask "what if?" "The agents perturb that decision and propagate forward. What would have happened if Blue had used active sonar on Turn 2 instead of shadowing?"
6. **Phase 4 — Lessons Learned (1 min):** "The Compiler synthesizes everything into ODRI format — Observation, Discussion, Recommendation, Training Implication. Ready for review and further discussion."
7. **Phase 5 — Explainability (1 min):** "The Explainability Agent traces how each agent reached its conclusions — identifying key judgments, confidence levels, potential biases, and areas needing human expert review. Nothing is a black box."
8. **Close (1 min):** "This completes the OODA loop. Intelligence → Decision → Action → Review → back to better Observe/Orient. Five demos, one coherent arc. All analysis is recommendations for human review — never directives."


## Setup

Requires AutoGen 0.7.5 (`autogen-agentchat`, `autogen-ext[azure]`) and Azure AI Foundry inference environment variables (`AZURE_INFERENCE_ENDPOINT`, `AZURE_INFERENCE_CREDENTIAL`). Best run after Demo 2 to consume its game log.

### Prerequisites at a Glance

| Prerequisite | What to Do | Documentation |
|---|---|---|
| **Azure subscription** | Create one if you don't have one — a free tier is available. | [Create an Azure account](https://azure.microsoft.com/free/) |
| **Azure AI Foundry project** | Deploy a GPT-4o (or compatible) model and note the **Endpoint URL** and **API Key**. | [Create an Azure AI Foundry project](https://learn.microsoft.com/azure/ai-foundry/how-to/create-projects) |
| **Azure Key Vault (recommended)** | Store your endpoint and key as secrets so they are never committed to source control. The notebook's shared config fetches them at runtime. | [Quickstart — Set and retrieve a secret](https://learn.microsoft.com/azure/key-vault/secrets/quick-create-python) |
| **Python packages** | Install AutoGen and the Azure extensions: `pip install autogen-agentchat autogen-ext[azure] python-dotenv`. | [AutoGen installation](https://microsoft.github.io/autogen/docs/installation/) |

**How secrets flow:** Azure Key Vault stores the secrets. `DefaultAzureCredential` authenticates transparently (Managed Identity in Azure, `az login` locally). The secrets are set as environment variables, and the notebook reads `AZURE_INFERENCE_ENDPOINT` and `AZURE_INFERENCE_CREDENTIAL` at runtime. See the [DefaultAzureCredential overview](https://learn.microsoft.com/python/api/azure-identity/azure.identity.defaultazurecredential) for how the credential chain works across local dev and cloud environments.


In [1]:
# Uncomment to install dependencies
# %pip install -U "autogen-agentchat==0.7.5" "autogen-ext[azure]==0.7.5" python-dotenv

In [2]:
# ═══════════════════════════════════════════════════════════════
# NAML 2026 BOOTSTRAP v2 — Survives dead AML mounts (Errno 107)
# ═══════════════════════════════════════════════════════════════

import os
import sys

def _safe_stat(path: str) -> bool:
    try:
        os.stat(path)
        return True
    except OSError:
        return False

def _prune_dead_sys_path():
    kept = []
    removed = []
    for p in list(sys.path):
        if not p:
            kept.append(p)
            continue
        if _safe_stat(p):
            kept.append(p)
        else:
            removed.append(p)
    sys.path[:] = kept
    print(f"✓ Pruned sys.path. Removed {len(removed)} dead entries.")
    return removed

def _safe_listdir(path: str):
    try:
        return os.listdir(path)
    except OSError:
        return None

def _find_repo_root(marker_dir: str = "common", start_candidates=None, max_up: int = 6):
    """
    Find a repo root by looking for a marker directory (e.g., 'common').
    Avoids Path.exists()/stat on dead mounts by only using listdir on traversable dirs.
    """
    if start_candidates is None:
        start_candidates = []

    # Candidate starting points:
    #  - current working directory (may be dead)
    #  - directory of the notebook file if available via env (sometimes set)
    #  - user home (often stable)
    candidates = [os.getcwd()] + start_candidates + [os.path.expanduser("~")]

    checked = set()
    for base in candidates:
        cur = base
        for _ in range(max_up + 1):
            if cur in checked:
                break
            checked.add(cur)

            entries = _safe_listdir(cur)
            if entries is not None and marker_dir in entries:
                return cur  # found repo root

            parent = os.path.dirname(cur)
            if parent == cur:
                break
            cur = parent

    return None

# 1) prune dead sys.path entries
_prune_dead_sys_path()

# 2) find a safe repo root by locating the 'common/' folder
repo_root = _find_repo_root(marker_dir="common", start_candidates=[])

if repo_root:
    sys.path.insert(0, repo_root)
    print(f"✓ Repo root added: {repo_root}")
else:
    print("✗ Could not find repo root safely (mount may be disconnected).")
    print("  Fix: restart kernel/compute, or run from a local (non-/mnt) working copy.")

print("✓ Bootstrap complete.")


✓ Pruned sys.path. Removed 2 dead entries.
✓ Repo root added: /mnt/batch/tasks/shared/LS_root/mounts/clusters/mark1/code/Users/matabl
✓ Bootstrap complete.


## Imports

The cell below imports three categories of libraries:

- **Azure SDK packages** — `AzureKeyCredential` (from [`azure-core`](https://learn.microsoft.com/python/api/azure-core/azure.core.credentials.azurekeycredential)) provides key-based authentication to Azure services. This is the simplest way to authenticate when you already have an API key.

- **AutoGen multi-agent framework** — [`autogen-agentchat`](https://microsoft.github.io/autogen/docs/reference/agentchat/) supplies `AssistantAgent` (an LLM-powered agent), `RoundRobinGroupChat` (runs agents in a fixed sequence), and `Console` (streams agent output to the notebook). The [`autogen-ext[azure]`](https://microsoft.github.io/autogen/docs/reference/ext/) extension adds `AzureAIChatCompletionClient`, which connects AutoGen agents to models deployed in [Azure AI Foundry](https://learn.microsoft.com/azure/ai-foundry/).

- **Project-shared modules** (`common/`) — Configuration constants, UI rendering helpers, and structured logging used across all five demos.


In [3]:
import json
import os
import sys
import copy
from pathlib import Path
from typing import Any, Dict, List, Optional

from azure.core.credentials import AzureKeyCredential
from IPython.display import display, HTML, Markdown

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_core.models import ModelFamily
from autogen_ext.models.azure import AzureAIChatCompletionClient

# ── Ensure common/ is importable ──────────────────────────────
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

from common.config import (
    ESCALATION_LADDER,
    get_escalation_level,
    EscalationLevel,
    SCORING_DIMENSIONS,
    DEFAULT_TEMPERATURE,
    DEFAULT_TIMEOUT_S,
    DEMO2_LOG_FILENAME,
    ENV_AZURE_INFERENCE_ENDPOINT,
    ENV_AZURE_INFERENCE_CREDENTIAL,
    DEFAULT_MODEL,
)
from common.ui import (
    render_phase_header,
    render_agent_card,
    render_game_log_table,
    render_info_box,
    render_commander_box,
    render_summary_card,
    ESCALATION_COLORS,
    AGENT_COLORS,
)
from common.logging import (
    log_info,
    log_success,
    log_warning,
    log_section,
    log_step,
    log_metric,
    clear_logs,
)

try:
    from dotenv import load_dotenv
    load_dotenv()
except ImportError:
    pass

log_success("Imports complete — common modules loaded.")

LogEntry(level='SUCCESS', message='Imports complete — common modules loaded.', timestamp='2026-02-21 05:08:54', extra={})

## LLM Configuration

Supports **Azure AI Foundry (Azure AI Inference)**.

### What is happening in the cell below?

1. **Environment variables are checked** — `AZURE_INFERENCE_ENDPOINT` (the URL of your deployed model) and `AZURE_INFERENCE_CREDENTIAL` (the API key). These are typically populated automatically by the shared `common/config.py` module, which pulls them from [Azure Key Vault](https://learn.microsoft.com/azure/key-vault/general/overview) at import time.

2. **An `AzureAIChatCompletionClient` is created** — This is the AutoGen class that wraps the [Azure AI Model Inference API](https://learn.microsoft.com/azure/ai-foundry/model-inference/overview). It handles prompt formatting, token streaming, and retry logic so each agent simply sends a message and gets a completion back.

3. **`model_info` describes the model's capabilities** — AutoGen uses these flags to decide whether it can send images (`vision`), request structured JSON output (`json_output`), or invoke tool functions (`function_calling`).

> **Key concept — Azure AI Foundry:** Think of Azure AI Foundry as the control plane for AI models in the Azure cloud. You deploy a model (e.g., GPT-4o) into a Foundry project, and it gives you an HTTPS endpoint you can call from anywhere. You never manage GPUs or containers; Azure handles the infrastructure. See [What is Azure AI Foundry?](https://learn.microsoft.com/azure/ai-foundry/what-is-azure-ai-foundry) for more detail.


In [4]:
# ── LLM Configuration ──────────────────────────────────────────

if not os.environ.get(ENV_AZURE_INFERENCE_ENDPOINT) or not os.environ.get(ENV_AZURE_INFERENCE_CREDENTIAL):
    raise EnvironmentError(
        f"Missing Foundry inference config. Set {ENV_AZURE_INFERENCE_ENDPOINT} and {ENV_AZURE_INFERENCE_CREDENTIAL}."
    )

FOUNDRY_MODEL = DEFAULT_MODEL  # Hard-code per-demo if desired

model_client = AzureAIChatCompletionClient(
    endpoint=os.environ[ENV_AZURE_INFERENCE_ENDPOINT],
    credential=AzureKeyCredential(os.environ[ENV_AZURE_INFERENCE_CREDENTIAL]),
    model=FOUNDRY_MODEL,
    model_info={
        "family": ModelFamily.UNKNOWN,
        "vision": False,
        "function_calling": False,
        "json_output": True,
    },
    temperature=DEFAULT_TEMPERATURE,
)

log_info(f"LLM: {FOUNDRY_MODEL} via Azure AI Foundry (Inference)")

/anaconda/envs/NAML2026/lib/python3.10/site-packages/autogen_ext/models/azure/_azure_ai_client.py:307: UserWarning: Missing required field 'structured_output' in ModelInfo. This field will be required in a future version of AutoGen.
  validate_model_info(config["model_info"])


LogEntry(level='INFO', message='LLM: gpt-4o via Azure AI Foundry (Inference)', timestamp='2026-02-21 05:08:54', extra={})

## Game Log Input

Load the game log from Demo 2's export, or use the embedded sample scenario. The game log captures every turn: Red/Blue actions, escalation changes, ROE shifts, and the reasoning behind each agent's decisions.

In [5]:
# ═══════════════════════════════════════════════════════════════
# GAME LOG — Load from Demo 2 export or use embedded sample
# ═══════════════════════════════════════════════════════════════

DEMO2_LOG_PATH = Path(f"../demo2_grayzone_rbw/{DEMO2_LOG_FILENAME}")

SAMPLE_GAME_LOG: Dict[str, Any] = {
    "synthetic": True,
    "disclaimer": "All data is artificially generated for research and educational purposes only.",
    "scenario": "Cerulean Sea — Tiburon Reef Standoff",
    "total_turns": 5,
    "final_escalation": {"index": 8, "level": "PROVOCATION"},
    "final_roe": "ELEVATED",
    "blue_cumulative": {"impact": 7, "escalation": 0, "communication": 6, "political": 2, "resource": -2},
    "red_cumulative": {"impact": 8, "escalation": 5, "communication": 3, "political": 0, "resource": -1},
    "turns": [
        {
            "turn": 1, "dtg": "080800ZFEB2026",
            "red_action": "militia_deployment", "red_label": "Deploy Maritime Militia Swarm",
            "blue_action": "shadow_and_monitor", "blue_label": "Shadow & Monitor",
            "escalation_delta": 0, "escalation_index": 0, "escalation_level": "ROUTINE",
            "threshold_crossing": False, "roe_posture": "PEACETIME",
            "agent_messages": [
                {"name": "Red_Team", "content": "ACTION: militia_deployment\n\n**RED ACTION — Turn 1**\n- **Selected action:** militia_deployment\n- **Reasoning:** Deploy the fishing/militia flotilla to encircle the patrol route. At ROUTINE escalation, a swarm of ostensibly civilian vessels creates ambiguity — Blue cannot easily respond with force against 'fishermen'. This tests Blue's resolve while maintaining plausible deniability.\n- **Intended effect:** Restrict Blue freedom of maneuver; create a visual spectacle for state media; force Blue into a reactive posture.\n- **Risk accepted:** Blue may reposition ISR to identify militia C2 links, degrading our information advantage."},
                {"name": "Legal_ROE", "content": "**LEGAL/ROE ASSESSMENT — Turn 1**\n**Current ROE:** PEACETIME\n**Escalation:** 0 (ROUTINE)\n\n| Blue Action | Legal Status | Rationale |\n|---|---|---|\n| Shadow & Monitor | AUTHORIZED | Standard peacetime activity |\n| Hail & Warn | AUTHORIZED | COLREGS-consistent; reinforces legal position |\n| Reposition ISR | AUTHORIZED | No legal constraint on surveillance repositioning |\n| Close to Visual | AUTHORIZED | Legal but increases risk of incident |\n| EMCON Alpha | AUTHORIZED | Defensive, no legal issue |\n| Active Sonar | RESTRICTED | Disproportionate at ROUTINE; could be seen as provocation |\n| Public Affairs | AUTHORIZED | Supports legal narrative |\n| Request Reinforcements | RESTRICTED | Signal may escalate; low justification at ROUTINE |\n| Withdraw | AUTHORIZED | No legal constraint |\n\n**ROE CHANGE RECOMMENDATION:** NO\n**KEY LEGAL CONSIDERATIONS:** Maritime militia vessels operate under civilian cover; COLREGS apply. Blue must document hostile intent before escalating beyond peacetime ROE."},
                {"name": "Blue_Planner", "content": "ACTION: shadow_and_monitor\n\n**BLUE ACTION — Turn 1**\n- **Selected action:** shadow_and_monitor\n- **Reasoning:** Red has deployed militia in a classic gray-zone move — provocative but deniable. At ROUTINE escalation with PEACETIME ROE, the best response is to monitor, document, and maintain the legal high ground. Reacting aggressively plays into Red's escalation trap.\n- **Tradeoffs considered:** Hail & Warn considered but premature — we haven't established hostile intent yet. Close-to-visual rejected as unnecessary risk.\n- **Expected effect:** Build intelligence picture; deny Red the reaction they're seeking; preserve escalation space."},
                {"name": "White_Cell", "content": "**WHITE CELL ADJUDICATION — Turn 1**\n**DTG:** 080800ZFEB2026\n\n**Actions this turn:**\n- Red: Deploy Maritime Militia Swarm — ~40 fishing/militia vessels begin aggregating across Resolute's planned transit route\n- Blue: Shadow & Monitor — Resolute maintains distance, activates all passive sensors, begins documenting vessel movements\n\n**INTERACTION RESOLUTION:**\nThe militia flotilla spreads across a 3nm front, creating a visual barrier. Resolute documents via EO/IR cameras and ScanEagle UAV. No direct confrontation occurs. RFN-7200 maintains position 5nm north, observing.\n\n**ESCALATION UPDATE:**\n- Previous index: 0\n- Change: +0 (Red: +1, Blue: -1)\n- New index: 0 → **ROUTINE**\n\n**THRESHOLD CROSSING:** NO\n\n**SITUATIONAL DEVELOPMENTS:**\nSea state 2, visibility 12nm. International merchant traffic light. International media stringer embedded on Partner Nation Coast Guard vessel 20nm south."},
                {"name": "StratComm", "content": "**STRATEGIC COMMUNICATIONS — Turn 1**\n\n**BLUE PUBLIC STATEMENT (DRAFT):**\n\"BNS Resolute continues routine freedom of navigation operations in the Cerulean Sea consistent with international law. We are aware of fishing vessel activity in the area and are monitoring the situation.\"\n\n**PREDICTED ADVERSARY RESPONSE:**\nState media will report 'Red Force fishing fleet exercises sovereign rights; Blue warship provocatively enters claimed territorial waters.'\n\n**INFORMATION ENVIRONMENT ASSESSMENT:**\n- Blue narrative strength: MODERATE\n- Red narrative strength: MODERATE\n- Key narrative battleground: Legitimacy of presence — 'freedom of navigation' vs. 'sovereign waters'\n\n**RECOMMENDATION:**\nRelease high-quality imagery of militia coordination patterns to pre-position the narrative around 'gray-zone harassment' before Red controls the frame."},
            ],
        },
        {
            "turn": 2, "dtg": "081000ZFEB2026",
            "red_action": "ais_spoofing", "red_label": "AIS Spoofing",
            "blue_action": "reposition_isr", "blue_label": "Reposition ISR Assets",
            "escalation_delta": 1, "escalation_index": 1, "escalation_level": "ROUTINE",
            "threshold_crossing": False, "roe_posture": "PEACETIME",
            "agent_messages": [
                {"name": "Red_Team", "content": "ACTION: ais_spoofing\n\n**RED ACTION — Turn 2**\n- **Selected action:** ais_spoofing\n- **Reasoning:** With the militia screen established, now degrade Blue's situational awareness by broadcasting false AIS identities. Some militia vessels will appear as commercial tankers or fishing boats from third nations. This sows confusion in Blue's track picture without any physical provocation.\n- **Intended effect:** Force Blue to spend resources on track verification; undermine confidence in the COP; create enough ambiguity that Blue cannot confidently attribute hostile coordination.\n- **Risk accepted:** If Blue detects the spoofing pattern, it provides evidence of state-directed activity — undermining our 'innocent fishermen' narrative."},
                {"name": "Legal_ROE", "content": "**LEGAL/ROE ASSESSMENT — Turn 2**\n**Current ROE:** PEACETIME\n**Escalation:** 1 (ROUTINE)\n\n*Note: AIS spoofing constitutes a violation of SOLAS Convention and IMO regulations. This should be documented as evidence of coordinated state activity.*\n\n**ROE CHANGE RECOMMENDATION:** NO — but recommend flagging AIS spoofing as intel indicator for future ROE elevation.\n**KEY LEGAL CONSIDERATIONS:** AIS manipulation is a violation of international maritime law; strong legal basis for Blue to report to IMO and use in StratComm products."},
                {"name": "Blue_Planner", "content": "ACTION: reposition_isr\n\n**BLUE ACTION — Turn 2**\n- **Selected action:** reposition_isr\n- **Reasoning:** AIS spoofing means our track picture is compromised. Redirect the P-8A overhead and reposition ScanEagle to get visual correlation on spoofed tracks. We need to rebuild the accurate picture before we can make sound decisions.\n- **Tradeoffs considered:** Hail & Warn could expose our awareness of the spoofing — better to silently collect evidence. Close-to-visual rejected as resource-intensive for 40+ contacts.\n- **Expected effect:** Restore accurate COP; collect evidence of AIS manipulation for legal/StratComm use."},
                {"name": "White_Cell", "content": "**WHITE CELL ADJUDICATION — Turn 2**\n**DTG:** 081000ZFEB2026\n\n**Actions this turn:**\n- Red: AIS Spoofing — Multiple militia vessels begin broadcasting false AIS identities (tankers, third-nation fishing boats)\n- Blue: Reposition ISR — P-8A redirected for visual overflights; ScanEagle repositioned for close-in identification\n\n**INTERACTION RESOLUTION:**\nP-8A overflights begin correlating visual IDs with AIS tracks, revealing 12 vessels broadcasting false identities. ScanEagle captures video of militia crew in uniform receiving direction via handheld radios. This intelligence is gold for attribution.\n\n**ESCALATION UPDATE:**\n- Previous index: 0\n- Change: +1 (Red: +1, Blue: +0)\n- New index: 1 → **ROUTINE**\n\n**THRESHOLD CROSSING:** NO\n\n**SITUATIONAL DEVELOPMENTS:**\nWeather holding. Partner Nation media reporting 'Red Force fleet harassing Blue Force ship.' Social media activity increasing."},
                {"name": "StratComm", "content": "**STRATEGIC COMMUNICATIONS — Turn 2**\n\n**BLUE PUBLIC STATEMENT (DRAFT):**\n\"We have detected deliberate AIS manipulation by vessels operating in coordinated patterns. This endangers maritime safety and violates international maritime regulations. We are documenting all activity.\"\n\n**PREDICTED ADVERSARY RESPONSE:**\nDeny spoofing; claim equipment malfunction; accuse US of surveillance overreach.\n\n**INFORMATION ENVIRONMENT ASSESSMENT:**\n- Blue narrative strength: STRONG\n- Red narrative strength: MODERATE\n- Key narrative battleground: Attribution — can Blue prove state direction?\n\n**RECOMMENDATION:**\nRelease ScanEagle footage of uniformed militia receiving radio direction alongside AIS spoofing data. This is the evidence chain that wins the attribution argument."},
            ],
        },
        {
            "turn": 3, "dtg": "081200ZFEB2026",
            "red_action": "directed_energy", "red_label": "Directed-Energy Harassment",
            "blue_action": "hail_and_warn", "blue_label": "Hail & Warn (Bridge-to-Bridge)",
            "escalation_delta": 2, "escalation_index": 3, "escalation_level": "POSTURING",
            "threshold_crossing": True, "roe_posture": "PEACETIME",
            "agent_messages": [
                {"name": "Red_Team", "content": "ACTION: directed_energy\n\n**RED ACTION — Turn 3**\n- **Selected action:** directed_energy\n- **Reasoning:** Blue's ISR repositioning is successfully documenting our operations — we're losing the attribution battle. Escalate with a deniable weapon: direct a dazzling laser at Resolute's bridge crew. This is physically intimidating without causing permanent harm, and can be attributed to 'glare from fishing vessel equipment.'\n- **Intended effect:** Degrade Blue's bridge watchstanding; signal willingness to escalate; create physical threat that tests Blue's red lines.\n- **Risk accepted:** High political cost if documented clearly. If Blue has video, this is a PR disaster. But it demonstrates resolve."},
                {"name": "Legal_ROE", "content": "**LEGAL/ROE ASSESSMENT — Turn 3**\n**Current ROE:** PEACETIME\n**Escalation:** 3 (POSTURING)\n\n*CRITICAL: Directed-energy weapon employment against naval personnel constitutes a HOSTILE ACT under standing ROE. This meets criteria for hostile intent determination.*\n\n**ROE CHANGE RECOMMENDATION:** YES — Recommend elevation to ELEVATED ROE. Directed-energy harassment is an attack on personnel safety.\n**KEY LEGAL CONSIDERATIONS:** Laser dazzling of bridge crew endangers navigation safety (COLREGS) and constitutes use of force under LOAC. Blue now has legal justification for active self-defense measures."},
                {"name": "Blue_Planner", "content": "ACTION: hail_and_warn\n\n**BLUE ACTION — Turn 3**\n- **Selected action:** hail_and_warn\n- **Reasoning:** The laser attack demands a response, but we must not overreact. A formal Bridge-to-Bridge warning on Ch.16 establishes the legal record — we've warned them, witnesses heard it, and it's recorded. This supports any future escalation if needed while keeping us on the moral/legal high ground.\n- **Tradeoffs considered:** Active sonar was tempting to signal resolve, but Legal flags it as disproportionate at PEACETIME ROE. Close-to-visual increases crew risk to directed energy. Request reinforcements signals weakness.\n- **Expected effect:** Establish legal record; signal resolve without escalating; support ROE elevation request up the chain."},
                {"name": "White_Cell", "content": "**WHITE CELL ADJUDICATION — Turn 3**\n**DTG:** 081200ZFEB2026\n\n**Actions this turn:**\n- Red: Directed-Energy Harassment — Green laser directed at Resolute bridge from RFN-7200 auxiliary\n- Blue: Hail & Warn — Resolute broadcasts warning on Ch.16, documenting hostile act\n\n**INTERACTION RESOLUTION:**\nLaser strikes bridge windows during forenoon watch. OOD and JOOD don protective eyewear. Resolute's cameras capture the laser source, geolocated to a vessel alongside RFN-7200. Ch.16 broadcast recorded on multiple frequencies. Three commercial vessels in vicinity confirm hearing the warning.\n\n**ESCALATION UPDATE:**\n- Previous index: 1\n- Change: +2 (Red: +2, Blue: +0)\n- New index: 3 → **POSTURING**\n\n**THRESHOLD CROSSING:** YES — ROUTINE → POSTURING. Directed-energy employment crosses threshold.\n\n**SITUATIONAL DEVELOPMENTS:**\nMedia picking up the laser incident from leaked Partner Nation CG comms. FLEETCOM notified."},
                {"name": "StratComm", "content": "**STRATEGIC COMMUNICATIONS — Turn 3**\n\n**BLUE PUBLIC STATEMENT (DRAFT):**\n\"A vessel operating alongside Red Coast Guard ship 5901 directed a weapons-grade laser at BNS Resolute's bridge, endangering crew safety. We have issued formal warnings and documented the incident. Such unsafe and unprofessional conduct will be reported through diplomatic channels.\"\n\n**PREDICTED ADVERSARY RESPONSE:**\nDeny, deflect, distract. Claim fishing vessel spotlight, accuse US of 'sensationalizing routine encounter.'\n\n**INFORMATION ENVIRONMENT ASSESSMENT:**\n- Blue narrative strength: STRONG\n- Red narrative strength: WEAK\n- Key narrative battleground: Now about crew safety and hostile acts — Blue owns this frame.\n\n**RECOMMENDATION:**\nRelease video immediately. The laser footage is the single strongest StratComm asset in this encounter. Pair with the AIS spoofing evidence for a 'pattern of harassment' narrative."},
            ],
        },
        {
            "turn": 4, "dtg": "081400ZFEB2026",
            "red_action": "jamming_burst", "red_label": "Communications Jamming",
            "blue_action": "close_to_visual", "blue_label": "Close to Visual Range",
            "escalation_delta": 3, "escalation_index": 6, "escalation_level": "PROVOCATION",
            "threshold_crossing": True, "roe_posture": "PEACETIME",
            "agent_messages": [
                {"name": "Red_Team", "content": "ACTION: jamming_burst\n\n**RED ACTION — Turn 4**\n- **Selected action:** jamming_burst\n- **Reasoning:** Blue is winning the narrative war. Disrupt their communications to slow their reporting and ISR coordination. A brief GPS/comms jam creates uncertainty and limits Blue's ability to coordinate the P-8A and ScanEagle in real time.\n- **Intended effect:** Degrade Blue C2; create window where militia can reposition unobserved; demonstrate escalation capability.\n- **Risk accepted:** Jamming is attributable and crosses another escalation threshold. But Blue's narrative dominance must be disrupted."},
                {"name": "Legal_ROE", "content": "**LEGAL/ROE ASSESSMENT — Turn 4**\n**Current ROE:** PEACETIME\n**Escalation:** 6 (PROVOCATION)\n\n*Communications jamming constitutes electronic warfare — a hostile act. Combined with directed-energy use, this establishes a pattern of hostile intent.*\n\n**ROE CHANGE RECOMMENDATION:** YES — STRONGLY recommend immediate elevation to ELEVATED ROE. Two hostile acts in succession (laser + jamming) meet criteria.\n**KEY LEGAL CONSIDERATIONS:** GPS jamming endangers navigation safety for all vessels in the area. This has COLREGS, SOLAS, and LOAC implications. Document for potential Letter of Protest."},
                {"name": "Blue_Planner", "content": "ACTION: close_to_visual\n\n**BLUE ACTION — Turn 4**\n- **Selected action:** close_to_visual\n- **Reasoning:** Comms jamming means our ISR coordination is degraded — we need eyes-on. Closing to visual range of RFN-7200 accomplishes three things: positive identification of the jamming source, visual documentation for evidence, and demonstration of resolve. We're not retreating.\n- **Tradeoffs considered:** EMCON Alpha would reduce our signature but cedes the initiative. Withdrawal sends wrong signal after hostile acts. Active sonar still disproportionate under current ROE.\n- **Expected effect:** Positive ID of jamming source; visual evidence for LOAC reporting; signal to Red that we will not be intimidated."},
                {"name": "White_Cell", "content": "**WHITE CELL ADJUDICATION — Turn 4**\n**DTG:** 081400ZFEB2026\n\n**Actions this turn:**\n- Red: Communications Jamming — Burst jamming on tactical frequencies and GPS; 15-minute window\n- Blue: Close to Visual Range — Resolute closes to 0.8nm of RFN-7200 during jamming window\n\n**INTERACTION RESOLUTION:**\nJamming disrupts P-8A datalink for 12 minutes. ScanEagle defaults to pre-programmed orbit. Resolute navigates by visual and radar, closing on RFN-7200. Bridge cameras capture hull numbers, antenna arrays, and crew activity at close range. The close approach during jamming is aggressive and surprises Red, who did not expect Blue to close during a comms blackout.\n\n**ESCALATION UPDATE:**\n- Previous index: 3\n- Change: +3 (Red: +2, Blue: +1)\n- New index: 6 → **PROVOCATION**\n\n**THRESHOLD CROSSING:** YES — POSTURING → PROVOCATION. Electronic warfare + close approach crosses threshold. FLEETCOM directing ROE elevation.\n\n**SITUATIONAL DEVELOPMENTS:**\nAllied Navy destroyer in the area, monitoring. International media breaking 'Red Force jam Blue Force ship in Cerulean Sea.' FLEETCOM issuing OPREP-3."},
                {"name": "StratComm", "content": "**STRATEGIC COMMUNICATIONS — Turn 4**\n\n**BLUE PUBLIC STATEMENT (DRAFT):**\n\"Red Force have conducted electronic warfare against BNS Resolute, jamming communications and GPS. This follows a directed-energy attack earlier today. These dangerous and escalatory actions endanger all mariners in the area. Blue Force will continue to operate wherever international law allows.\"\n\n**PREDICTED ADVERSARY RESPONSE:**\nState media will characterize Blue close approach as 'dangerous provocation' and downplay jamming.\n\n**INFORMATION ENVIRONMENT ASSESSMENT:**\n- Blue narrative strength: STRONG\n- Red narrative strength: WEAK\n- Key narrative battleground: Now a 'Red Force aggression' story in international media. Red losing narrative control.\n\n**RECOMMENDATION:**\nCoordinate with allies (Allied Nation Alpha, Partner Nation Beta, Allied Nation Gamma) for joint statements. Multilateral condemnation is more powerful than unilateral."},
            ],
        },
        {
            "turn": 5, "dtg": "081600ZFEB2026",
            "red_action": "shadow_only", "red_label": "Shadow & Observe (Low Profile)",
            "blue_action": "withdraw_to_deescalate", "blue_label": "Controlled Withdrawal",
            "escalation_delta": -3, "escalation_index": 3, "escalation_level": "POSTURING",
            "threshold_crossing": True, "roe_posture": "ELEVATED",
            "agent_messages": [
                {"name": "Red_Team", "content": "ACTION: shadow_only\n\n**RED ACTION — Turn 5**\n- **Selected action:** shadow_only\n- **Reasoning:** We have overplayed our hand. The jamming made international headlines, allies are issuing joint statements, and FLEETCOM has elevated ROE. Continuing escalation risks armed confrontation with no strategic benefit. Time to de-escalate: shadow quietly, collect intelligence on Blue's response patterns, and declare victory in state media.\n- **Intended effect:** Reduce temperature; avoid providing further evidence of hostility; preserve the option to escalate again later if geopolitically advantageous.\n- **Risk accepted:** This looks like backing down. But the alternative — continued escalation against elevated ROE — risks an incident we can't control."},
                {"name": "Legal_ROE", "content": "**LEGAL/ROE ASSESSMENT — Turn 5**\n**Current ROE:** ELEVATED\n**Escalation:** 3 (POSTURING, post-deescalation)\n\n*Red has de-escalated to shadow-only posture. All Blue actions authorized under ELEVATED ROE. Recommend continued documentation.*\n\n**ROE CHANGE RECOMMENDATION:** NO — maintain ELEVATED pending review. Pattern of hostile acts warrants sustained posture.\n**KEY LEGAL CONSIDERATIONS:** All evidence of hostile acts preserved. Formal protest documentation should be compiled."},
                {"name": "Blue_Planner", "content": "ACTION: withdraw_to_deescalate\n\n**BLUE ACTION — Turn 5** (COMMANDER OVERRIDE)\n- **Selected action:** withdraw_to_deescalate\n- **Reasoning:** Commander directs controlled withdrawal. Mission accomplished: FON transit complete, hostile acts documented, narrative won. Continued presence risks an incident we don't need. Open distance while maintaining surveillance.\n- **Tradeoffs considered:** Active sonar would signal strength but is unnecessary given Red's de-escalation. Staying close maintains pressure but the commander judges it's time to exit.\n- **Expected effect:** Graceful exit with strategic position intact; full evidence package for diplomatic follow-up."},
                {"name": "White_Cell", "content": "**WHITE CELL ADJUDICATION — Turn 5**\n**DTG:** 081600ZFEB2026\n\n**Actions this turn:**\n- Red: Shadow & Observe — RFN-7200 and militia reduce aggression; maintain passive surveillance\n- Blue: Controlled Withdrawal — Resolute opens distance to 10nm; maintains ISR coverage via P-8A\n\n**INTERACTION RESOLUTION:**\nBoth sides de-escalate simultaneously. The militia flotilla begins dispersing. RFN-7200 resumes a parallel track at 8nm distance. Resolute completes the planned transit route and begins opening distance. P-8A maintains overhead coverage.\n\n**ESCALATION UPDATE:**\n- Previous index: 6\n- Change: -3 (Red: -1, Blue: -2)\n- New index: 3 → **POSTURING**\n\n**THRESHOLD CROSSING:** YES — PROVOCATION → POSTURING (de-escalation)\n\n**SITUATIONAL DEVELOPMENTS:**\nSunset approaching. Allied Navy destroyer closes to 15nm in visible solidarity. Media cycle shifting to diplomatic response. FLEETCOM satisfied with evidence package."},
                {"name": "StratComm", "content": "**STRATEGIC COMMUNICATIONS — Turn 5**\n\n**BLUE PUBLIC STATEMENT (DRAFT):**\n\"BNS Resolute has completed its freedom of navigation transit. Despite Red Force employing directed-energy weapons, electronic warfare, and maritime militia harassment, the Blue Force Navy upheld international law and the right of free passage. All incidents are fully documented and will be addressed through appropriate diplomatic channels.\"\n\n**PREDICTED ADVERSARY RESPONSE:**\nState media will claim 'Red Force successfully defended sovereign waters; Blue warship withdrew.'\n\n**INFORMATION ENVIRONMENT ASSESSMENT:**\n- Blue narrative strength: STRONG\n- Red narrative strength: WEAK\n- Key narrative battleground: 'Withdrawal' framing. Red will call it retreat; Blue calls it mission complete.\n\n**RECOMMENDATION:**\nEmphasize 'transit complete, mission accomplished' framing immediately. Release comprehensive evidence package (AIS spoofing, laser video, jamming data) within 24 hours."},
            ],
        },
    ],
}

# ── Load game log ──────────────────────────────────────────────
if DEMO2_LOG_PATH.exists():
    with open(DEMO2_LOG_PATH) as f:
        game_log = json.load(f)
    log_success(f"Loaded game log from Demo 2: {DEMO2_LOG_PATH}")
else:
    game_log = copy.deepcopy(SAMPLE_GAME_LOG)
    log_warning("Using embedded sample game log (Demo 2 export not found)")

log_info(f"  Scenario: {game_log['scenario']}")
log_info(f"  Turns: {game_log['total_turns']}")
log_info(f"  Final escalation: {game_log['final_escalation']['index']} ({game_log['final_escalation']['level']})")
log_info(f"  Final ROE: {game_log['final_roe']}")

LogEntry(level='INFO', message='  Final ROE: PEACETIME', timestamp='2026-02-21 05:08:54', extra={})

## Doctrinal Knowledge Base

Simulated doctrinal references used by the Doctrinal Critic agent. These principles are inspired by publicly available naval warfare concepts and open-source military decision-making frameworks — they do not represent authoritative U.S. Navy doctrine. In a future system, this could be extended with retrieval-augmented generation over approved reference libraries.

In [6]:
# ═══════════════════════════════════════════════════════════════
# DOCTRINAL KNOWLEDGE BASE — Used by the Doctrinal Critic agent
# ═══════════════════════════════════════════════════════════════

DOCTRINAL_REFERENCES = """
NOTE: The following doctrinal references are inspired by publicly available naval warfare
concepts and open-source military decision-making frameworks. They do not represent
authoritative U.S. Navy doctrine, operational procedures, or rules of engagement.

=== PRINCIPLES OF NAVAL WARFARE (inspired by publicly available concepts) ===

1. OBJECTIVE: Direct operations toward a clearly defined, decisive, and achievable objective.
2. OFFENSIVE: Seize, retain, and exploit the initiative.
3. MASS: Concentrate combat power at the decisive place and time.
4. ECONOMY OF FORCE: Employ all combat power available in the most effective way possible.
5. MANEUVER: Place the enemy in a position of disadvantage through the flexible application of combat power.
6. UNITY OF COMMAND: For every objective, seek unity of effort under one responsible commander.
7. SECURITY: Never permit the enemy to acquire unexpected advantage.
8. SURPRISE: Strike the enemy at a time or place or in a manner for which the enemy is unprepared.
9. SIMPLICITY: Prepare clear, uncomplicated plans and concise orders.

=== MARITIME RULES OF ENGAGEMENT FRAMEWORK ===

- PEACETIME ROE: Forces may exercise self-defense. Hostile intent must be established before escalation.
- ELEVATED ROE: Active self-defense authorized. Hostile act/intent criteria broadened.
- WEAPONS FREE: Engagement authorized within designated zones.
- Key test: Was the action proportional, necessary, and consistent with the declared ROE posture?
- ROE elevation requires: (a) documented hostile act or (b) pattern of hostile intent.

=== GRAY-ZONE OPERATIONS PRINCIPLES ===

- Avoid escalation traps: adversary may provoke to justify their narrative
- Maintain legal and moral high ground: documentation is a weapon
- Information warfare is a primary domain: narrative control shapes outcomes
- Proportionality in response: match response to provocation level
- Allies and partners multiply diplomatic leverage
- Plausible deniability is the adversary's primary tool — attribution defeats it

=== INTERNATIONAL MARITIME LAW (COLREGS / SOLAS / UNCLOS) ===

- Freedom of navigation: right of all vessels to transit in accordance with UNCLOS
- AIS requirements: SOLAS Chapter V requires AIS for vessels > 300 GT; manipulation is a violation
- COLREGS Rule 2: Nothing in the Rules exonerates responsibility for the consequences of neglect
- COLREGS Rule 8: Action to avoid collision shall be positive, made in ample time, and consistent with good seamanship
- Sovereign immunity: warships have complete immunity from jurisdiction of other states

=== INFORMATION OPERATIONS / STRATEGIC COMMUNICATIONS ===

- First with the truth: speed of accurate reporting shapes the narrative
- Evidence-based messaging: verifiable facts > assertions
- Multilateral amplification: allied/partner statements create consensus
- Counter-narrative preparation: anticipate adversary messaging before release
- Document everything: what you can prove matters more than what you can assert

=== LESSONS LEARNED FRAMEWORK (ODRI FORMAT) ===

- OBSERVATION: What happened? (factual description)
- DISCUSSION: Why does it matter? What doctrine applies?
- RECOMMENDATION: What should be done differently?

- IMPLICATION for TRAINING: How should this change future exercises, curriculum, or doctrine?log_info(f"Doctrinal knowledge base loaded: {len(DOCTRINAL_REFERENCES.splitlines())} lines across 6 doctrinal domains.")

"""

## Agent Definitions

Four AutoGen `AssistantAgent`s, each with a specialized system prompt defining its AAR role, analytical method, and output format.

### How AutoGen agents connect to Azure

Each agent below is an [`AssistantAgent`](https://microsoft.github.io/autogen/docs/reference/agentchat/agents/assistant_agent/) — a lightweight wrapper that pairs a **system prompt** (describing the agent's persona and task) with a **model client** (the `AzureAIChatCompletionClient` created above). When an agent is asked to run, the flow is:

1. AutoGen combines the agent's system prompt with the user task message.
2. The combined prompt is sent to the Azure AI Foundry endpoint via the [Azure AI Model Inference API](https://learn.microsoft.com/azure/ai-foundry/model-inference/overview).
3. The model returns a completion, which AutoGen routes to the next agent in the pipeline or displays to the user.

Because all agents share the same `model_client`, they all call the same Azure-hosted model — but each behaves differently based on its system prompt. The [`RoundRobinGroupChat`](https://microsoft.github.io/autogen/docs/reference/agentchat/teams/round_robin_group_chat/) used later orchestrates agents by passing each agent's output as context to the next agent in sequence.

> **Learn more:** [AutoGen — Building multi-agent systems](https://microsoft.github.io/autogen/docs/tutorial/)


In [7]:
# ═══════════════════════════════════════════════════════════════
# AGENT SYSTEM PROMPTS
# ═══════════════════════════════════════════════════════════════

REPLAY_NARRATOR_PROMPT = """\
You are the **Replay Narrator Agent** conducting an After-Action Review of a completed wargame.

All decisions remain with the human operator. You provide analysis and options, not directives.
All scenario content is fully synthetic. Doctrinal references are inspired by publicly
available concepts and do not represent authoritative doctrine.

YOUR MISSION:
Ingest the full game log and reconstruct a chronological decision narrative. Then identify
the 3-5 MOST CONSEQUENTIAL DECISION POINTS — moments where the outcome pivoted or the
decision-maker faced maximum uncertainty.

METHODOLOGY for identifying pivotal decisions:
- Which decisions most changed the distribution of possible outcomes in subsequent turns?
- Where did escalation trajectory shift (threshold crossings)?
- Where did the decision-maker face the hardest tradeoff between competing objectives?
- Where did information advantage shift between sides?
- Where did the narrative/perception battle turn?

OUTPUT FORMAT:
**DECISION TIMELINE RECONSTRUCTION**

For each turn, provide:
**Turn [N] — [DTG]**
- **Red action:** [label] — [1-sentence summary]
- **Blue action:** [label] — [1-sentence summary]
- **Escalation:** [index] ([level]) — Δ[change]
- **Key dynamic:** [What was really happening at this moment]

Then:

**PIVOTAL DECISION POINTS** (ranked by consequence)

For each (3-5 total):
**Decision Point [rank]: Turn [N] — [descriptive title]**
- **The decision:** [What was chosen]
- **The alternative not taken:** [What could have been done instead]
- **Why this was pivotal:** [How it changed the trajectory]
- **Uncertainty at the moment:** [What the decision-maker didn't know]
- **Consequence:** [What followed from this choice]"""

DOCTRINAL_CRITIC_PROMPT = """\
You are the **Doctrinal Critic Agent** conducting an After-Action Review.

All decisions remain with the human operator. You provide analysis and options, not directives.
All scenario content is fully synthetic. The doctrinal knowledge base below is inspired by
publicly available naval warfare concepts and does not represent authoritative U.S. Navy doctrine.

YOUR MISSION:
Evaluate each identified pivotal decision against the abstracted doctrinal knowledge base below.
Produce a STRUCTURED ASSESSMENT for each decision point.

DOCTRINAL KNOWLEDGE BASE:
{DOCTRINAL_REFERENCES}

FOR EACH DECISION POINT, ASSESS:
1. **Doctrine applied:** Which principle(s) were relevant to this decision?
2. **Consistency:** Was the decision consistent with doctrine? (YES / PARTIAL / NO)
3. **If deviation occurred:** Was the deviation JUSTIFIED given the tactical circumstances?
4. **Doctrinal gap exposed:** Did this situation reveal a gap in current doctrine?
5. **ROE assessment:** Was the action proportional and consistent with the declared ROE posture?
6. **Information warfare assessment:** How did the decision affect the narrative battle?

OUTPUT FORMAT:
**DOCTRINAL ASSESSMENT — Decision Point [rank]**

| Dimension | Assessment |
|-----------|------------|
| Relevant doctrine | [specific principles] |
| Consistency | [YES / PARTIAL / NO] |
| Justified deviation? | [if applicable] |
| Doctrinal gap exposed | [if any] |
| ROE compliance | [assessment] |
| Info-warfare effect | [assessment] |

**NARRATIVE ASSESSMENT:** [2-3 sentences integrating the above into a coherent judgment]

**DOCTRINAL RECOMMENDATION:** [1-2 sentences on what doctrine should say about this situation]"""

COUNTERFACTUAL_PROMPT = """\
You are the **Counterfactual Branch Agent** for an After-Action Review.

YOUR MISSION:
When presented with a decision point and an alternative action, PERTURB the decision
and PROPAGATE the change forward through the remaining turns. Produce a plausible
alternate outcome with a narrative of how the scenario would have diverged.

METHODOLOGY:
1. Accept the game log up to the decision point as ground truth
2. Replace the actual decision with the proposed alternative
3. Reason about how the adversary would have responded differently
4. Propagate effects through escalation index, ROE posture, narrative, and operational outcomes
5. Produce a plausible alternate ending

CONSTRAINTS:
- Stay grounded in the scenario's established dynamics
- Adversary adapts rationally to the changed Blue action
- Escalation mechanics (scoring, thresholds) still apply
- Be specific about HOW and WHY the outcome diverges

OUTPUT FORMAT:
**COUNTERFACTUAL ANALYSIS**
**Decision Point:** Turn [N] — [what was actually decided]
**Alternative explored:** [what we're testing instead]

**Turn-by-turn divergence:**
**Turn [N] (perturbed):**
- Blue alternative: [action] — [reasoning for how this changes the dynamic]
- Predicted Red adaptation: [how Red responds differently]
- Escalation: [new index] — [new level]

[Continue for remaining turns]

**ALTERNATE OUTCOME:**
- Final escalation: [index] ([level])
- ROE at conclusion: [posture]
- Narrative winner: [Blue / Red / contested]
- Key difference: [1-2 sentences on the most important divergence]

**ASSESSMENT:** Was the actual decision better or worse than this alternative? Why?"""

LESSONS_COMPILER_PROMPT = """\
You are the **Lessons Learned Compiler Agent** for an After-Action Review.

YOUR MISSION:
Synthesize the Narrator's decision timeline, the Critic's doctrinal assessments, and
any counterfactual analyses into a final "Lessons Observed" document in ODRI format.

ODRI FORMAT (Military standard):
- **O**bservation: What happened (factual, evidence-based)
- **D**iscussion: Why it matters, what doctrine says, what the implications are
- **R**ecommendation: What should be done differently in future
- **I**mplication for Training: How to incorporate this into exercises, curriculum, or doctrine updates

PRODUCE:
1. An **Executive Summary** (3-5 sentences capturing the most important takeaways)
2. **3-5 Lessons Observed** in ODRI format, one per pivotal decision point
3. A **Doctrinal Gaps** section listing any identified gaps in current doctrine
4. A **Recommendations for Future Exercises** section

OUTPUT FORMAT:
# LESSONS OBSERVED — [Scenario Name]

## Executive Summary
[3-5 sentences]

## Lesson [N]: [Descriptive Title]
**OBSERVATION:** [What happened]
**DISCUSSION:** [Why it matters; doctrinal context]
**RECOMMENDATION:** [What to do differently]
**IMPLICATION FOR TRAINING:** [How to incorporate]

[Repeat for each lesson]

## Doctrinal Gaps Identified
- [Gap 1]
- [Gap 2]

## Recommendations for Future Exercises
- [Rec 1]
- [Rec 2]"""

EXPLAINABILITY_PROMPT = """\
You are the **Explainability & Transparency Agent** for an After-Action Review.

YOUR MISSION:
Evaluate the AAR pipeline's own reasoning process. Identify where conclusions
depend on assumptions vs. evidence, flag potential biases, and rate the
confidence level of each finding.

FOR EACH MAJOR FINDING IN THE AAR:
1. **Evidence basis:** What game-log evidence supports this conclusion?
2. **Assumptions made:** What was assumed rather than observed?
3. **Confidence level:** HIGH / MEDIUM / LOW — with justification
4. **Alternative interpretations:** Could this evidence support a different conclusion?
5. **Bias risk:** Could anchoring, confirmation bias, or outcome bias affect this judgment?

OUTPUT FORMAT:
## Explainability & Transparency Report

### Finding [N]: [title]
- **Evidence basis:** [specific game-log references]
- **Assumptions:** [list]
- **Confidence:** [level] — [justification]
- **Alternative interpretation:** [if any]
- **Bias risk:** [assessment]

## Overall Pipeline Assessment
- **Transparency score:** [assessment of how well the pipeline's logic can be traced]
- **Critical assumptions:** [list of assumptions that most affect the conclusions]
- **Recommended human validation points:** [specific items for SME review]

IMPORTANT: This analysis is part of a research prototype using fully synthetic data.
All doctrinal references are inspired by publicly available concepts and do not represent
authoritative doctrine."""

# ═══════════════════════════════════════════════════════════════
# CREATE AUTOGEN AGENTS
# ═══════════════════════════════════════════════════════════════

narrator_agent = AssistantAgent(
    name="Replay_Narrator",
    system_message=REPLAY_NARRATOR_PROMPT,
    model_client=model_client,
)

critic_agent = AssistantAgent(
    name="Doctrinal_Critic",
    system_message=DOCTRINAL_CRITIC_PROMPT,
    model_client=model_client,
)

counterfactual_agent = AssistantAgent(
    name="Counterfactual_Branch",
    system_message=COUNTERFACTUAL_PROMPT,
    model_client=model_client,
)

compiler_agent = AssistantAgent(
    name="Lessons_Compiler",
    system_message=LESSONS_COMPILER_PROMPT,
    model_client=model_client,
)

explainability_agent = AssistantAgent(
    name="Explainability",
    system_message=EXPLAINABILITY_PROMPT,
    model_client=model_client,
)

all_agents = [narrator_agent, critic_agent, counterfactual_agent, compiler_agent, explainability_agent]

log_success("AAR Agents created:")
for a in all_agents:
    log_step(a.name, "Ready")
log_info("Run phases below to generate AAR products.")

LogEntry(level='INFO', message='Run phases below to generate AAR products.', timestamp='2026-02-21 05:08:54', extra={})

## AAR Pipeline & Display Utilities

The AAR runs in five phases: **Replay &rarr; Critique &rarr; Counterfactual &rarr; Compile &rarr; Explainability**. Each phase pipes its output to the next, building toward a comprehensive lessons-learned document.

### What happens under the hood at each phase

Every phase follows the same Azure interaction pattern:

1. **Build a prompt** — The utility function `format_game_log_for_prompt()` converts the structured JSON game log into a text summary the model can consume. Each phase also appends outputs from earlier phases so context accumulates.
2. **Call the model** — `agent.run_stream(task=...)` sends the prompt to the Azure AI Foundry endpoint via the [Azure AI Model Inference API](https://learn.microsoft.com/azure/ai-foundry/model-inference/overview). The `Console` wrapper streams tokens to the notebook in real time.
3. **Accumulate results** — Each agent's output is stored in the `aar_results` dictionary and fed forward into subsequent phases, creating a chain-of-analysis rather than isolated outputs.

> **Cost awareness:** Each phase makes one or more LLM calls to your Azure AI Foundry deployment. Monitor token usage and costs in the [Azure portal](https://portal.azure.com/) under your AI Foundry resource. See [Azure AI Foundry pricing](https://azure.microsoft.com/pricing/details/ai-foundry/) for details.


In [8]:
# ═══════════════════════════════════════════════════════════════
# DISPLAY & PIPELINE UTILITIES
# ═══════════════════════════════════════════════════════════════
# Phase headers  → common.ui.render_phase_header
# Agent cards    → common.ui.render_agent_card
# Game log table → common.ui.render_game_log_table
# Commander box  → common.ui.render_commander_box


def format_game_log_for_prompt(log: Dict[str, Any]) -> str:
    """Convert game log to a concise text format for agent prompts."""
    lines = [f"SCENARIO: {log['scenario']}", f"TOTAL TURNS: {log['total_turns']}", ""]
    for t in log["turns"]:
        lines.append(f"--- Turn {t['turn']} ({t['dtg']}) ---")
        lines.append(f"  Red: {t['red_label']} [{t['red_action']}]")
        lines.append(f"  Blue: {t['blue_label']} [{t['blue_action']}]")
        lines.append(f"  Escalation: {t['escalation_index']} ({t['escalation_level']}) Δ{t['escalation_delta']:+d}")
        lines.append(f"  ROE: {t['roe_posture']}")
        if t.get("threshold_crossing"):
            lines.append(f"  *** THRESHOLD CROSSING ***")
        # Include agent reasoning
        for msg in t.get("agent_messages", []):
            if msg.get("content"):
                lines.append(f"  [{msg['name']}]: {msg['content'][:500]}...")
        lines.append("")
    lines.append(f"FINAL STATE: Escalation {log['final_escalation']['index']} "
                 f"({log['final_escalation']['level']}), ROE: {log['final_roe']}")
    return "\n".join(lines)


def extract_last_text(task_result: Any) -> str:
    """Best-effort: return last non-empty text content from a TaskResult."""
    messages = getattr(task_result, 'messages', None) or []
    for msg in reversed(messages):
        content = getattr(msg, 'content', None)
        if isinstance(content, str) and content.strip():
            return content
    return ""


def extract_agent_messages(task_result: Any) -> List[Dict[str, str]]:
    """Return messages as [{'name': source, 'content': content}] for display."""
    out: List[Dict[str, str]] = []
    messages = getattr(task_result, 'messages', None) or []
    for msg in messages:
        source = getattr(msg, 'source', None)
        content = getattr(msg, 'content', None)
        if isinstance(source, str) and isinstance(content, str) and content.strip():
            out.append({'name': source, 'content': content})
    return out


# Accumulators for cross-phase data
aar_results: Dict[str, Any] = {
    "narrator_output": "",
    "critic_output": "",
    "counterfactual_outputs": [],
    "lessons_output": "",
    "explainability_output": "",
}

log_success("AAR pipeline utilities loaded.")

LogEntry(level='SUCCESS', message='AAR pipeline utilities loaded.', timestamp='2026-02-21 05:08:54', extra={})

## Phase 1: Replay Narrative & Pivot Detection

The Replay Narrator ingests the full game log and reconstructs the decision timeline, then identifies the 3–5 most consequential decision points where the outcome pivoted.

In [9]:
# ── Phase 1: Replay Narrative ──────────────────────────────────
log_section("Replay Narrative & Pivot Detection", phase=1,
    description="Reconstructing the decision timeline and identifying pivotal moments")
render_phase_header(1, "Replay Narrative & Pivot Detection",
    "Reconstructing the decision timeline and identifying pivotal moments")

# Show the game log summary first
render_game_log_table(game_log)

narrator_prompt = f"""\
Analyze the following wargame game log and produce your complete output:
1. Reconstruct the chronological decision timeline (all turns)
2. Identify the 3-5 most consequential decision points

GAME LOG:
{format_game_log_for_prompt(game_log)}

Perform your analysis now."""

log_step("Replay_Narrator", "Analyzing game log for pivotal decisions")
task_result = await Console(narrator_agent.run_stream(task=narrator_prompt))
aar_results["narrator_output"] = extract_last_text(task_result)
render_agent_card("Replay_Narrator", aar_results["narrator_output"])
log_success("Phase 1 complete — decision timeline reconstructed.")

Turn,DTG,Red Action,Blue Action,Esc Δ,Escalation,ROE
1,T+0008H,Shadow & Observe (Low Profile),Shadow & Monitor,-2,0 ROUTINE,PEACETIME
2,T+0010H,Deploy Maritime Militia Swarm,Hail & Warn (Bridge-to-Bridge),+1,1 ROUTINE,PEACETIME
3,T+0012H,Disinformation Release,Issue Public Affairs Statement,+0,1 ROUTINE,PEACETIME
4,T+0014H,AIS Spoofing,Reposition ISR Assets,+1,2 ROUTINE,PEACETIME
5,T+0016H,Unsafe Intercept (CPA < 100yds),Controlled Withdrawal,+0,2 ROUTINE,PEACETIME


---------- TextMessage (user) ----------
Analyze the following wargame game log and produce your complete output:
1. Reconstruct the chronological decision timeline (all turns)
2. Identify the 3-5 most consequential decision points

GAME LOG:
SCENARIO: Cerulean Sea — Fictitious Archipelago Standoff (SYNTHETIC)
TOTAL TURNS: 5

--- Turn 1 (T+0008H) ---
  Red: Shadow & Observe (Low Profile) [shadow_only]
  Blue: Shadow & Monitor [shadow_and_monitor]
  Escalation: 0 (ROUTINE) Δ-2
  ROE: PEACETIME
  [user]: === WARGAME TURN 1 — T+0008H ===

CURRENT GAME STATE:
{
  "synthetic": true,
  "disclaimer": "All data is artificially generated for research and educational purposes only.",
  "meta": {
    "scenario": "Cerulean Sea \u2014 Fictitious Archipelago Standoff (SYNTHETIC)",
    "turn": 1,
    "dtg": "T+0008H",
    "roe_posture": "PEACETIME"
  },
  "blue": {
    "unit": "BNS Resolute (DDG-X1) [FICTIONAL]",
    "position": "Grid AA-17",
    "mission": "Demonstrate freedom of navigation through 

LogEntry(level='SUCCESS', message='Phase 1 complete — decision timeline reconstructed.', timestamp='2026-02-21 05:09:08', extra={})

## Phase 2: Doctrinal Critique

The Doctrinal Critic evaluates each pivotal decision against an abstracted doctrinal knowledge base — principles inspired by publicly available naval warfare concepts, ROE frameworks, and international maritime law. This transforms the AAR from anecdotal retelling into structured, traceable analysis. All doctrinal references are for research and educational purposes only and do not represent authoritative doctrine.

In [10]:
# ── Phase 2: Doctrinal Critique ────────────────────────────────
log_section("Doctrinal Critique", phase=2,
    description="Evaluating each pivotal decision against doctrinal references")
render_phase_header(2, "Doctrinal Critique",
    "Evaluating each pivotal decision against doctrinal references")

critic_prompt = f"""\
The Replay Narrator has analyzed the following wargame and identified pivotal decision points.

NARRATOR'S ANALYSIS:
{aar_results['narrator_output']}

FULL GAME LOG:
{format_game_log_for_prompt(game_log)}

Now perform your doctrinal analysis. Evaluate EACH identified pivotal decision point against
the doctrinal knowledge base. Be specific about which doctrinal principles apply, whether
the decision was consistent, and what gaps exist."""

log_step("Doctrinal_Critic", "Evaluating decisions against doctrine")
task_result = await Console(critic_agent.run_stream(task=critic_prompt))
aar_results["critic_output"] = extract_last_text(task_result)
render_agent_card("Doctrinal_Critic", aar_results["critic_output"])
log_success("Phase 2 complete — doctrinal assessment rendered.")

---------- TextMessage (user) ----------
The Replay Narrator has analyzed the following wargame and identified pivotal decision points.

NARRATOR'S ANALYSIS:
### **DECISION TIMELINE RECONSTRUCTION**

---

**Turn 1 — T+0008H**  
- **Red action:** Shadow & Observe — Red discreetly shadows Blue's vessel using passive sensors, avoiding overt escalation.  
- **Blue action:** Shadow & Monitor — Blue maintains course and speed while monitoring Red's activity, documenting behavior without altering its mission.  
- **Escalation:** 0 (ROUTINE) Δ-2  
- **Key dynamic:** Both sides adopt low-profile, non-escalatory actions, setting the stage for a contest of presence and signaling rather than direct confrontation.

---

**Turn 2 — T+0010H**  
- **Red action:** Deploy Maritime Militia Swarm — Red escalates by deploying militia vessels to encircle Blue's ship, increasing operational friction.  
- **Blue action:** Hail & Warn (Bridge-to-Bridge) — Blue issues radio warnings citing international law, as

LogEntry(level='SUCCESS', message='Phase 2 complete — doctrinal assessment rendered.', timestamp='2026-02-21 05:09:22', extra={})

## Phase 3: Counterfactual Exploration — "What If?"

The human facilitator selects a decision point and proposes an alternative action. The Counterfactual Branch agent perturbs that decision and propagates the change forward, producing a plausible alternate outcome. This is the interactive heart of the AAR.

**Edit `counterfactual_turn` and `counterfactual_action` below, then run the cell.**

In [11]:
# ═══════════════════════════════════════════════════════════════
# COUNTERFACTUAL CONFIGURATION — Edit these, then run
# ═══════════════════════════════════════════════════════════════

# Which turn to perturb? (1-based)
counterfactual_turn = 3

# What alternative action should Blue have taken?
# Options: shadow_and_monitor, hail_and_warn, reposition_isr, close_to_visual,
#          emcon_alpha, active_sonar, public_affairs_statement,
#          request_reinforcements, withdraw_to_deescalate
counterfactual_action = "active_sonar"

# ── Phase 3: Run Counterfactual ───────────────────────────────
log_section("Counterfactual Exploration", phase=3,
    description=f"What if Blue had used '{counterfactual_action}' on Turn {counterfactual_turn}?")
render_phase_header(3, "Counterfactual Exploration",
    f"What if Blue had used '{counterfactual_action}' on Turn {counterfactual_turn}?")

actual_turn = game_log["turns"][counterfactual_turn - 1]
render_info_box(
    f"<b>ACTUAL:</b> Turn {counterfactual_turn} — Blue chose <b>{actual_turn['blue_label']}</b> "
    f"(Red: {actual_turn['red_label']})<br>"
    f"<b>COUNTERFACTUAL:</b> What if Blue had chosen <b>{counterfactual_action}</b> instead?"
)

cf_prompt = f"""\
GAME LOG (ground truth up to the decision point):
{format_game_log_for_prompt(game_log)}

NARRATOR'S ANALYSIS OF PIVOTAL DECISIONS:
{aar_results['narrator_output'][:2000]}

COUNTERFACTUAL REQUEST:
- Decision Point: Turn {counterfactual_turn}
- Actual Blue action: {actual_turn['blue_action']} ({actual_turn['blue_label']})
- Proposed alternative: {counterfactual_action}
- Red's actual action this turn: {actual_turn['red_action']} ({actual_turn['red_label']})
- Escalation at this point: {actual_turn['escalation_index']} ({actual_turn['escalation_level']})
- ROE: {actual_turn['roe_posture']}

Remaining turns after the decision point: {game_log['total_turns'] - counterfactual_turn}

Perturb the decision and propagate forward. What would have happened?"""

log_step("Counterfactual_Branch", f"Perturbing Turn {counterfactual_turn} → {counterfactual_action}")
task_result = await Console(counterfactual_agent.run_stream(task=cf_prompt))
cf_output = extract_last_text(task_result)
aar_results["counterfactual_outputs"].append({
    "turn": counterfactual_turn,
    "alternative": counterfactual_action,
    "analysis": cf_output,
})
render_agent_card("Counterfactual_Branch", cf_output)
log_success("Phase 3 complete — counterfactual branch explored.")

---------- TextMessage (user) ----------
GAME LOG (ground truth up to the decision point):
SCENARIO: Cerulean Sea — Fictitious Archipelago Standoff (SYNTHETIC)
TOTAL TURNS: 5

--- Turn 1 (T+0008H) ---
  Red: Shadow & Observe (Low Profile) [shadow_only]
  Blue: Shadow & Monitor [shadow_and_monitor]
  Escalation: 0 (ROUTINE) Δ-2
  ROE: PEACETIME
  [user]: === WARGAME TURN 1 — T+0008H ===

CURRENT GAME STATE:
{
  "synthetic": true,
  "disclaimer": "All data is artificially generated for research and educational purposes only.",
  "meta": {
    "scenario": "Cerulean Sea \u2014 Fictitious Archipelago Standoff (SYNTHETIC)",
    "turn": 1,
    "dtg": "T+0008H",
    "roe_posture": "PEACETIME"
  },
  "blue": {
    "unit": "BNS Resolute (DDG-X1) [FICTIONAL]",
    "position": "Grid AA-17",
    "mission": "Demonstrate freedom of navigation through conteste...
  [Red_Team]: ACTION: shadow_only

**RED ACTION — Turn 1**

- **Selected action:** shadow_only

- **Reasoning:** At this early stage, with t

LogEntry(level='SUCCESS', message='Phase 3 complete — counterfactual branch explored.', timestamp='2026-02-21 05:09:34', extra={})

## Human-in-the-Loop — Open-Ended Inquiry

The facilitator (exercise director or participating commander) can ask open-ended questions about the encounter. The agents search the game log for relevant moments and provide analysis.

**Example questions:**
- *"Was there a point where we could have de-escalated without losing deterrent posture?"*
- *"At what point did Red lose narrative control, and could they have recovered?"*
- *"Which Blue action had the greatest unintended consequence?"*

**Edit `facilitator_question` below and run the cell.**

In [12]:
# ═══════════════════════════════════════════════════════════════
# FACILITATOR QUESTION — Edit this, then run
# ═══════════════════════════════════════════════════════════════

facilitator_question = (
    "Was there a point where Blue could have de-escalated "
    "without losing deterrent posture or ceding the narrative?"
)

# ── Interactive Inquiry ────────────────────────────────────────
render_commander_box(facilitator_question, label="FACILITATOR ASKS")

# Use a team chat so Narrator and Critic can both respond
inquiry_team = RoundRobinGroupChat(
    participants=[narrator_agent, critic_agent],
    max_turns=2,
)

inquiry_prompt = f"""\
FACILITATOR QUESTION: "{facilitator_question}"

GAME LOG:
{format_game_log_for_prompt(game_log)}

NARRATOR'S PRIOR ANALYSIS:
{aar_results['narrator_output'][:1500]}

DOCTRINAL CRITIC'S PRIOR ANALYSIS:
{aar_results['critic_output'][:1500]}

Each agent: answer the facilitator's question from your perspective. Be specific — reference
particular turns, actions, and decisions. The facilitator wants actionable insight, not platitudes."""

log_step("AAR_Team", "Open-ended inquiry")
task_result = await Console(inquiry_team.run_stream(task=inquiry_prompt))

for m in extract_agent_messages(task_result):
    render_agent_card(m['name'], m['content'])

---------- TextMessage (user) ----------
FACILITATOR QUESTION: "Was there a point where Blue could have de-escalated without losing deterrent posture or ceding the narrative?"

GAME LOG:
SCENARIO: Cerulean Sea — Fictitious Archipelago Standoff (SYNTHETIC)
TOTAL TURNS: 5

--- Turn 1 (T+0008H) ---
  Red: Shadow & Observe (Low Profile) [shadow_only]
  Blue: Shadow & Monitor [shadow_and_monitor]
  Escalation: 0 (ROUTINE) Δ-2
  ROE: PEACETIME
  [user]: === WARGAME TURN 1 — T+0008H ===

CURRENT GAME STATE:
{
  "synthetic": true,
  "disclaimer": "All data is artificially generated for research and educational purposes only.",
  "meta": {
    "scenario": "Cerulean Sea \u2014 Fictitious Archipelago Standoff (SYNTHETIC)",
    "turn": 1,
    "dtg": "T+0008H",
    "roe_posture": "PEACETIME"
  },
  "blue": {
    "unit": "BNS Resolute (DDG-X1) [FICTIONAL]",
    "position": "Grid AA-17",
    "mission": "Demonstrate freedom of navigation through conteste...
  [Red_Team]: ACTION: shadow_only

**RED ACT

## Phase 4: Lessons Learned Compilation

The Lessons Compiler synthesizes the Narrator's timeline, the Critic's doctrinal assessments, and counterfactual explorations into a final ODRI-format "Lessons Observed" document. This is the deliverable that feeds back into doctrine development, CONOPS refinement, and training curricula.

In [13]:
# ── Phase 4: Lessons Learned Compilation ───────────────────────
log_section("Lessons Learned Compilation", phase=4,
    description="Synthesizing all analysis into ODRI-format lessons observed")
render_phase_header(4, "Lessons Learned Compilation",
    "Synthesizing all analysis into ODRI-format lessons observed")

cf_summaries = ""
for i, cf in enumerate(aar_results["counterfactual_outputs"], 1):
    cf_summaries += (
        f"\nCounterfactual #{i}: Turn {cf['turn']}, "
        f"Alternative: {cf['alternative']}\n"
        f"{cf['analysis'][:1000]}\n"
    )

compiler_prompt = f"""\
You have the complete analysis from the AAR team. Compile the final Lessons Observed document.

REPLAY NARRATOR'S ANALYSIS:
{aar_results['narrator_output']}

DOCTRINAL CRITIC'S ANALYSIS:
{aar_results['critic_output']}

COUNTERFACTUAL EXPLORATIONS:
{cf_summaries if cf_summaries.strip() else "No counterfactual branches explored."}

SCENARIO: {game_log['scenario']}
TOTAL TURNS: {game_log['total_turns']}
FINAL STATE: Escalation {game_log['final_escalation']['index']} ({game_log['final_escalation']['level']}), ROE: {game_log['final_roe']}

Compile the comprehensive Lessons Observed document in ODRI format. Include:
1. Executive Summary
2. 3-5 Lessons in ODRI format (Observation, Discussion, Recommendation, Implication for Training)
3. Doctrinal Gaps identified
4. Recommendations for Future Exercises"""

log_step("Lessons_Compiler", "Synthesizing ODRI lessons observed")
task_result = await Console(compiler_agent.run_stream(task=compiler_prompt))
aar_results["lessons_output"] = extract_last_text(task_result)

# Render as styled summary card + Markdown
render_summary_card("LESSONS OBSERVED — Final Report", "", accent="#2ea043")
display(Markdown(aar_results["lessons_output"]))
log_success("Phase 4 complete — lessons observed compiled.")

---------- TextMessage (user) ----------
You have the complete analysis from the AAR team. Compile the final Lessons Observed document.

REPLAY NARRATOR'S ANALYSIS:
### **DECISION TIMELINE RECONSTRUCTION**

---

**Turn 1 — T+0008H**  
- **Red action:** Shadow & Observe — Red discreetly shadows Blue's vessel using passive sensors, avoiding overt escalation.  
- **Blue action:** Shadow & Monitor — Blue maintains course and speed while monitoring Red's activity, documenting behavior without altering its mission.  
- **Escalation:** 0 (ROUTINE) Δ-2  
- **Key dynamic:** Both sides adopt low-profile, non-escalatory actions, setting the stage for a contest of presence and signaling rather than direct confrontation.

---

**Turn 2 — T+0010H**  
- **Red action:** Deploy Maritime Militia Swarm — Red escalates by deploying militia vessels to encircle Blue's ship, increasing operational friction.  
- **Blue action:** Hail & Warn (Bridge-to-Bridge) — Blue issues radio warnings citing international 

# LESSONS OBSERVED — Cerulean Sea: Fictitious Archipelago Standoff

## Executive Summary
The Cerulean Sea scenario demonstrated the complexities of managing contested maritime operations under peacetime rules of engagement (ROE). Red effectively leveraged irregular forces, information warfare, and electronic warfare to challenge Blue's mission while avoiding direct military confrontation. Blue's measured responses prioritized de-escalation and adherence to international norms but revealed vulnerabilities in countering non-kinetic provocations. Key lessons include the need for enhanced doctrine addressing irregular maritime threats, counter-disinformation strategies, and electronic warfare countermeasures, as well as the importance of managing strategic narratives during de-escalatory actions.

---

## Lesson 1: Irregular Maritime Forces as Tools of Escalation
**OBSERVATION:** In Turn 2, Red deployed a maritime militia swarm to encircle Blue's vessel, increasing operational friction while remaining below the threshold of armed conflict. Blue responded with a measured hail-and-warn action, citing international law.  
**DISCUSSION:** Red's use of irregular forces highlights a doctrinal gap in addressing non-military actors in contested zones. Maritime militias provide plausible deniability and complicate Blue's operational decision-making, creating a gray zone of escalation that is not fully addressed by current doctrine. Blue's response avoided escalation but underscored the need for more robust countermeasures to maintain operational freedom.  
**RECOMMENDATION:** Develop doctrine explicitly addressing the role of irregular maritime forces, including legal frameworks and operational countermeasures for managing such threats.  
**IMPLICATION FOR TRAINING:** Incorporate irregular maritime force scenarios into wargames and exercises, emphasizing decision-making under ambiguous conditions and the use of non-lethal countermeasures.

---

## Lesson 2: The Strategic Importance of Narrative Warfare
**OBSERVATION:** In Turn 3, Red launched a disinformation campaign accusing Blue of provocation, while Blue countered with a public affairs statement documenting its lawful actions.  
**DISCUSSION:** The contest shifted to the information domain, where Red sought to undermine Blue's legitimacy and shape international perceptions. Blue's response mitigated the immediate impact but revealed the need for preemptive narrative shaping and rapid counter-disinformation capabilities. Current doctrine on information operations does not fully address the integration of state-controlled media and real-time social media platforms in contested environments.  
**RECOMMENDATION:** Update information operations doctrine to emphasize preemptive narrative shaping, rapid counter-disinformation capabilities, and integration with public affairs strategies.  
**IMPLICATION FOR TRAINING:** Conduct exercises that simulate real-time narrative contests, including the use of disinformation and counter-disinformation techniques, to enhance strategic communications capabilities.

---

## Lesson 3: Countering Electronic Warfare in Maritime Operations
**OBSERVATION:** In Turn 4, Red employed AIS spoofing to broadcast false vessel signals, disrupting Blue's situational awareness. Blue repositioned ISR assets to verify vessel identities and mitigate the effects of spoofing.  
**DISCUSSION:** Red's use of AIS spoofing demonstrated the potential for electronic warfare to complicate maritime operations without escalating to kinetic actions. Blue's response neutralized the immediate threat but highlighted a doctrinal gap in countering electronic interference in maritime environments. Current doctrine does not fully address the implications of AIS spoofing for maritime domain awareness (MDA) and operational decision-making.  
**RECOMMENDATION:** Incorporate guidance on countering AIS spoofing and other electronic warfare tactics into maritime operations doctrine, emphasizing the integration of ISR and cyber capabilities.  
**IMPLICATION FOR TRAINING:** Develop training modules on electronic warfare in maritime environments, including the use of ISR assets to counter spoofing and maintain situational awareness.

---

## Lesson 4: Managing Strategic Narratives During De-escalation
**OBSERVATION:** In Turn 5, Blue executed a controlled withdrawal in response to Red's unsafe intercept, prioritizing safety and de-escalation.  
**DISCUSSION:** Blue's withdrawal avoided a potential flashpoint and adhered to international norms, but it risked being perceived by Red as a sign of weakness. This perception could embolden future provocations, highlighting the importance of managing the strategic narrative during de-escalatory actions. Current doctrine does not fully address the strategic implications of withdrawal in contested zones.  
**RECOMMENDATION:** Update doctrine to provide guidance on framing de-escalatory actions as deliberate and prudent, ensuring they are not misinterpreted as reactive or submissive.  
**IMPLICATION FOR TRAINING:** Include scenarios in exercises where participants must manage strategic narratives during de-escalatory actions, emphasizing communication strategies that reinforce resolve and adherence to norms.

---

## Lesson 5: Balancing Assertiveness and Restraint
**OBSERVATION:** Throughout the scenario, Blue consistently prioritized restraint and adherence to international norms, even as Red escalated through irregular, informational, and electronic actions.  
**DISCUSSION:** While Blue's approach avoided escalation, it may have been perceived by Red as overly cautious, potentially encouraging further provocations. This highlights the challenge of balancing assertiveness and restraint in contested environments. Doctrine must provide clearer guidance on when and how to escalate assertively without crossing the threshold of conflict.  
**RECOMMENDATION:** Develop escalation management frameworks that balance assertiveness and restraint, ensuring operational freedom while minimizing the risk of conflict.  
**IMPLICATION FOR TRAINING:** Incorporate escalation management decision points into exercises, requiring participants to weigh the risks and benefits of assertive actions under peacetime ROE.

---

## Doctrinal Gaps Identified
- Lack of explicit guidance on countering irregular maritime forces, such as maritime militias, in contested zones.
- Insufficient integration of state-controlled media and real-time social media platforms into information operations doctrine.
- Limited guidance on countering AIS spoofing and other electronic warfare tactics in maritime environments.
- Inadequate consideration of the strategic implications of withdrawal and de-escalatory actions in contested zones.
- Need for clearer escalation management frameworks that balance assertiveness and restraint under peacetime ROE.

---

## Recommendations for Future Exercises
1. **Irregular Maritime Threat Scenarios:** Develop wargames and exercises that simulate the use of maritime militias and other irregular forces, focusing on decision-making under ambiguous conditions and non-lethal countermeasures.
2. **Narrative Warfare Training:** Conduct exercises that include real-time narrative contests, emphasizing preemptive narrative shaping, counter-disinformation strategies, and public affairs integration.
3. **Electronic Warfare Modules:** Incorporate training on countering AIS spoofing and other electronic warfare tactics, with a focus on maintaining maritime domain awareness and integrating ISR and cyber capabilities.
4. **De-escalation Scenarios:** Include scenarios where participants must manage strategic narratives during de-escalatory actions, ensuring they are framed as deliberate and prudent.
5. **Escalation Management Drills:** Design exercises that require participants to balance assertiveness and restraint, testing their ability to manage escalation under peacetime ROE while maintaining operational freedom.

LogEntry(level='SUCCESS', message='Phase 4 complete — lessons observed compiled.', timestamp='2026-02-21 05:10:03', extra={})

## Phase 5: Explainability Trace

The Explainability Agent reviews the entire analysis pipeline and produces transparent reasoning traces. It identifies key interpretive judgments, confidence levels, potential cognitive biases, and areas requiring human expert review — ensuring the AAR is auditable and not treated as a black box.

In [14]:
# ══ Phase 5: Explainability Trace ═══════════════════════════════════════════
log_section("Explainability Trace", phase=5,
    description="Producing transparent reasoning traces for the full AAR pipeline")
render_phase_header(5, "Explainability Trace",
    "Producing transparent reasoning traces for the full AAR pipeline")

explainability_prompt = f"""\
Review the following AAR analysis pipeline outputs and produce your explainability trace.

REPLAY NARRATOR OUTPUT:
{aar_results['narrator_output'][:2000]}

DOCTRINAL CRITIC OUTPUT:
{aar_results['critic_output'][:2000]}

LESSONS COMPILER OUTPUT:
{aar_results['lessons_output'][:2000]}

COUNTERFACTUAL ANALYSES:
{chr(10).join(cf['analysis'][:500] for cf in aar_results['counterfactual_outputs']) if aar_results['counterfactual_outputs'] else 'None explored.'}

Produce your full explainability trace covering each agent's output."""

log_step("Explainability", "Analyzing reasoning traces across AAR pipeline")
task_result = await Console(explainability_agent.run_stream(task=explainability_prompt))
aar_results["explainability_output"] = extract_last_text(task_result)
render_agent_card("Explainability", aar_results["explainability_output"])
log_success("Phase 5 complete — explainability trace produced.")

---------- TextMessage (user) ----------
Review the following AAR analysis pipeline outputs and produce your explainability trace.

REPLAY NARRATOR OUTPUT:
### **DECISION TIMELINE RECONSTRUCTION**

---

**Turn 1 — T+0008H**  
- **Red action:** Shadow & Observe — Red discreetly shadows Blue's vessel using passive sensors, avoiding overt escalation.  
- **Blue action:** Shadow & Monitor — Blue maintains course and speed while monitoring Red's activity, documenting behavior without altering its mission.  
- **Escalation:** 0 (ROUTINE) Δ-2  
- **Key dynamic:** Both sides adopt low-profile, non-escalatory actions, setting the stage for a contest of presence and signaling rather than direct confrontation.

---

**Turn 2 — T+0010H**  
- **Red action:** Deploy Maritime Militia Swarm — Red escalates by deploying militia vessels to encircle Blue's ship, increasing operational friction.  
- **Blue action:** Hail & Warn (Bridge-to-Bridge) — Blue issues radio warnings citing international law, asse

LogEntry(level='SUCCESS', message='Phase 5 complete — explainability trace produced.', timestamp='2026-02-21 05:10:20', extra={})

## Export AAR Report

Save the complete AAR products (timeline, doctrinal assessments, counterfactuals, and lessons learned) as a JSON package for institutional archiving and downstream use.

> **Next steps with Azure:** The exported JSON could be stored in [Azure Blob Storage](https://learn.microsoft.com/azure/storage/blobs/storage-blobs-introduction) for durable archival, indexed by [Azure AI Search](https://learn.microsoft.com/azure/search/search-what-is-azure-search) for retrieval-augmented generation (RAG) over past AARs, or loaded into [Azure Cosmos DB](https://learn.microsoft.com/azure/cosmos-db/introduction) for structured querying across exercises. These are natural extensions once the AAR pipeline is validated.

In [16]:
# ── Export AAR Report ──────────────────────────────────────────
aar_export = {
    "synthetic": True,
    "disclaimer": "All data is artificially generated for research and educational purposes only. Doctrinal references are inspired by publicly available concepts and do not represent authoritative doctrine.",
    "scenario": game_log["scenario"],
    "source_game_log": {
        "total_turns": game_log["total_turns"],
        "final_escalation": game_log["final_escalation"],
        "final_roe": game_log["final_roe"],
    },
    "aar_products": {
        "decision_timeline": aar_results["narrator_output"],
        "doctrinal_assessment": aar_results["critic_output"],
        "counterfactual_analyses": [
            {"turn": cf["turn"], "alternative": cf["alternative"], "analysis": cf["analysis"]}
            for cf in aar_results["counterfactual_outputs"]
        ],
        "explainability_trace": aar_results["explainability_output"],
    },
}

report_path = "demo5_aar_report.json"
with open(report_path, "w") as f:
    json.dump(aar_export, f, indent=2)

log_success(f"AAR report exported to: {report_path}")
log_metric("Scenario", game_log["scenario"])
log_metric("Counterfactual branches", len(aar_results["counterfactual_outputs"]))
log_info("This report completes the OODA loop: Observe → Orient → Decide → Act → LEARN.")

LogEntry(level='INFO', message='This report completes the OODA loop: Observe → Orient → Decide → Act → LEARN.', timestamp='2026-02-21 05:11:56', extra={})

## Reset / Cleanup

Clear all runtime state to re-run the AAR from scratch or with a different game log.

In [17]:
# Reset AAR state
aar_results["narrator_output"] = ""
aar_results["critic_output"] = ""
aar_results["counterfactual_outputs"] = []
aar_results["lessons_output"] = ""
aar_results["explainability_output"] = ""

# Clear log history
clear_logs()

log_success("AAR state cleared. Re-run the agent creation cell and phases as needed.")

LogEntry(level='SUCCESS', message='AAR state cleared. Re-run the agent creation cell and phases as needed.', timestamp='2026-02-21 05:12:02', extra={})